In [1]:
import pandahouse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import hashlib
import swifter
from scipy import stats
from scipy.stats import norm, ttest_ind
from tqdm import tqdm
%matplotlib inline

In [2]:
connection = {'host': 'https://clickhouse.lab.karpov.courses',
'database':'simulator_20241020',
'user':'student',
'password':'dpo_python_2020'
}

In [18]:
q1 = """
select views, count() as users
from (select  
    user_id,
    sum(action = 'view') as views
from {db}.feed_actions
where toDate(time) between '2024-09-20' and '2024-09-26'
group by user_id
)
group by views
order by views
"""

In [19]:
views = pandahouse.read_clickhouse(q1, connection=connection)

In [20]:
views['p'] = views['users']/views.users.sum()

In [27]:
q2 = """
select 
   ctr, count() as users
from (select toDate(time) as dt, 
    user_id,
    sum(action = 'like')/sum(action = 'view') as ctr
from {db}.feed_actions
where dt between '2024-09-20' and '2024-09-26'
group by dt, user_id
)
group by ctr
"""

In [28]:
ctr = pandahouse.read_clickhouse(q2, connection=connection)

In [29]:
ctr['p'] = ctr['users']/ctr.users.sum()

In [30]:
rng = np.random.default_rng()

In [42]:
q3 = """
select views, count() as users
from (select  
    user_id,
    sum(action = 'view') as views
from {db}.feed_actions
where toDate(time) between '2024-09-27' and '2024-10-03'
group by user_id
)
group by views
"""

In [57]:
view_I = pandahouse.read_clickhouse(q3, connection=connection)

In [59]:
view_I.users.sum()

49814

In [55]:
view_II.users.sum()

49814

In [53]:
q4 = """
select views, count(distinct user_id) as users
from (select  
    user_id,
    sum(action = 'view') as views
from {db}.feed_actions
where toDate(time) between '2024-09-20' and '2024-10-03'
group by user_id
)
group by views
"""

In [54]:
view_II = pandahouse.read_clickhouse(q3, connection=connection)

In [48]:
b = []
n = 30591
for _ in tqdm(range(20000)):
    views1 = rng.choice(views.views, size = n, replace = True, p = views.p).astype('int64')
    views1 = ((1 + rng.binomial(1, 0.5, n)) * rng.binomial(1, 0.9, size = n)) * (views1 >= 30) + views1
    views2 = rng.choice(views.views, size = n, replace = True, p = views.p).astype('int64')
    ctr1 = rng.choice(ctr.ctr, size = n, p = ctr.p, replace = True)
    ctr2 = rng.choice(ctr.ctr, size = n, p = ctr.p, replace = True)
    likes1 = rng.binomial(views1, ctr1)
    likes2 = rng.binomial(views2, ctr2)
    b.append((stats.ttest_ind(likes1, likes2, equal_var = False).pvalue < 0.05))

100%|██████████| 20000/20000 [07:40<00:00, 43.45it/s]


In [49]:
q = 0
for i in range(len(b)):
    if b[i] == True:
        q += 1
q / 20000 * 100

56.93

In [ ]:
#(здесь мы симулируем данные просмотров, CTR и лайков)

#mask_A = group_A_views >= 30
#mask_B = group_B_views >= 30

#scipy.stats.ttest_ind(group_A_likes[mask_A], group_B_likes[mask_B], equal_var=False)

In [61]:
l = []
n = 30591
for _ in tqdm(range(20000)):
    views1 = rng.choice(views.views, size = n, replace = True, p = views.p).astype('int64')
    views1 = ((1 + rng.binomial(1, 0.5, n)) * rng.binomial(1, 0.9, size = n)) * (views1 >= 30) + views1
    views2 = rng.choice(views.views, size = n, replace = True, p = views.p).astype('int64')
    ctr1 = rng.choice(ctr.ctr, size = n, p = ctr.p, replace = True)
    ctr2 = rng.choice(ctr.ctr, size = n, p = ctr.p, replace = True)
    likes1 = rng.binomial(views1, ctr1)
    likes2 = rng.binomial(views2, ctr2)
    l.append((stats.ttest_ind(likes1[views1 >= 30], likes2[views2 >= 30], equal_var = False).pvalue < 0.05))

100%|██████████| 20000/20000 [07:50<00:00, 42.52it/s]


In [62]:
q = 0
for i in range(len(l)):
    if l[i] == True:
        q += 1
q / 20000 * 100

65.10000000000001

In [65]:
q5 = """
SELECT uniq(user_id)
FROM {db}.feed_actions
where toDate(time) between '2024-09-20' and '2024-10-03'
"""

In [66]:
how = pandahouse.read_clickhouse(q5, connection=connection)

In [68]:
how/2

,uniq(user_id)
0,30590.5
